In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from sympy import (symbols, simplify)
from sympy.physics.mechanics import dynamicsymbols, init_vprinting
from sympy.physics.mechanics import Lagrangian, ReferenceFrame, Point, Particle,inertia, RigidBody, angular_momentum

In [ ]:
from optibot.symbolic import lagrange, diff_to_symb, SimpLagrangesMethod
from optibot.numpy import unpack

In [ ]:
fixed = {
    0:0.,
    2:0.,
    4:0.,
    5:0.,
    6:1.,
}

In [ ]:
_a = [{} for j in range(7)]

for ii in fixed:
    _a[ii]['q']=fixed[ii]
    
_a

In [ ]:
init_vprinting()

In [ ]:
from optibot.robots import Panda_Simp

In [ ]:
panda = Panda_Simp(replacedict_list=_a)

In [ ]:
plt.style.use('default')

In [ ]:
from optibot.opti import Opti_Problem
import time

In [ ]:
t0 = time.time()
print('start time:',time.strftime('%H:%M:%S ', time.localtime(t0)))

for ii in range(1000000):
    _ = ii**2**0.9232


t1 = time.time()
print('end time:',time.strftime('%H:%M:%S ', time.localtime(t1)))
dt = t1-t0
print('dt: ',dt, time.strftime('%H:%M:%S ',time.gmtime(dt)))

#### Warning: next cell can take several minutes to run (the first time)

In [ ]:
t0 = time.time()
print('start time:',time.strftime('%H:%M:%S ', time.localtime(t0)))

N = 25
verbose = True
silent = False
ini_guess = 'lin'
t_end = 2
params = []
scheme = 'hs_parab'

_opti = Opti_Problem(
    LM=panda,
    params = params,
    scheme = scheme,
    ini_guess= ini_guess,
    t_end = t_end,
    verbose=verbose,
    silent=silent,
)
_opti.dynamic_setup()
_opti.opti_setup(N)

t1 = time.time()
print('end time:',time.strftime('%H:%M:%S ', time.localtime(t1)))
dt = t1-t0
print('dt: ',dt, time.strftime('%H:%M:%S ',time.gmtime(dt)))

In [ ]:
_opti.apply_scheme()

In [ ]:
_opti.opti_points

In [ ]:
panda.robot.qlim

In [ ]:
pose_0 = np.array([0.,  1., 0., -1., 0., 0., 1. ])
pose_1 = np.array([0., -1., 0., -2., 0., 0., 1. ])

In [ ]:
np.expand_dims(pose_0, 0).shape

In [ ]:
pose_0.shape

In [ ]:
q_s = _opti.opti_points['q_s']
q_e = _opti.opti_points['q_e']
v_s = _opti.opti_points['v_s']
v_e = _opti.opti_points['v_e']

_opti.opti.subject_to(q_s == np.expand_dims(pose_0, 0))
_opti.opti.subject_to(q_e == np.expand_dims(pose_1, 0))

_opti.opti.subject_to(v_s == 0)
_opti.opti.subject_to(v_e == 0)

_opti.initial_guess(pose_0, pose_1)

In [ ]:
_opti.u_sq_cost()

In [ ]:
q_arr = _opti.opti_arrs['q']
q_arr_c = _opti.opti_arrs['q_c']
v_arr = _opti.opti_arrs['v']
v_arr_c = _opti.opti_arrs['v_c']
u_arr = _opti.opti_arrs['u']
u_arr_c = _opti.opti_arrs['u_c']
q_arr.shape

In [ ]:
panda.robot.qlim

In [ ]:
qlim = panda.robot.qlim
for ii in fixed:
    qlim[:,ii] = fixed[ii]
qlim

In [ ]:
q_dot_max = np.zeros(7)
q_dot_max[0:4] = 2.1750
q_dot_max[4:7] = 2.6100
q_dot_max

In [ ]:
tau_max = np.zeros(7)
tau_max[0:4] = 87
tau_max[4:7] = 12
tau_max

In [ ]:
tau_max = np.zeros(2)
tau_max[:] = 87
tau_max

In [ ]:
for ii in range(7):
    _opti.opti.subject_to(qlim[:,ii][0] <= q_arr[:,ii])
    _opti.opti.subject_to(qlim[:,ii][1] >= q_arr[:,ii])
    _opti.opti.subject_to(qlim[:,ii][0] <= q_arr_c[:,ii])
    _opti.opti.subject_to(qlim[:,ii][1] >= q_arr_c[:,ii])
    
    _opti.opti.subject_to(-q_dot_max[ii] <= v_arr[:,ii])
    _opti.opti.subject_to( q_dot_max[ii] >= v_arr[:,ii])
    _opti.opti.subject_to(-q_dot_max[ii] <= v_arr_c[:,ii])
    _opti.opti.subject_to( q_dot_max[ii] >= v_arr_c[:,ii])
    
for ii in range(2):
    _opti.opti.subject_to(-tau_max[ii] <= u_arr[:,ii])
    _opti.opti.subject_to( tau_max[ii] >= u_arr[:,ii])
    _opti.opti.subject_to(-tau_max[ii] <= u_arr_c[:,ii])
    _opti.opti.subject_to( tau_max[ii] >= u_arr_c[:,ii])

In [ ]:
t0 = time.time()
print('start time:',time.strftime('%H:%M:%S ', time.localtime(t0)))

_opti.chrono_solve(1)

t1 = time.time()
print('end time:',time.strftime('%H:%M:%S ', time.localtime(t1)))
dt = t1-t0
print('dt: ',dt, time.strftime('%H:%M:%S ',time.gmtime(dt)))

In [ ]:
_opti.results['cpudt'],time.strftime('%H:%M:%S ',time.gmtime(_opti.results['cpudt']))

In [ ]:
%matplotlib inline

In [ ]:
plt.figure(figsize=[16,10])
plt.plot(_opti.results['t'],_opti.results['q'],marker='o')
plt.plot(_opti.results['t_c'],_opti.results['q_c'],'o')
plt.grid()

In [ ]:
plt.figure(figsize=[16,10])
plt.plot(_opti.results['t'],_opti.results['v'],marker='o')
plt.plot(_opti.results['t_c'],_opti.results['v_c'],'o')
plt.grid()

In [ ]:
plt.figure(figsize=[16,10])
plt.plot(_opti.results['t'],_opti.results['u'],marker='o')
plt.plot(_opti.results['t_c'],_opti.results['u_c'],'o')
plt.grid()

In [ ]:
from roboticstoolbox.models.DH import Panda as PD

In [ ]:
pd = PD()

In [ ]:
pd

In [ ]:
pd.fkine(_opti.results['q'])

In [ ]:
%matplotlib notebook
%config NotebookBackend.figure_format = 'retina'

In [ ]:
pd.plot(_opti.results['q'])